# 📖 [LMX Tutorial] Absolute and Relative motion

Refer to the following document:

http://download.movensys.com:8222/doc/wmx3/en/html/api/page_WMXDOC_TUTORIAL_SEC3_1_ABS_REL.html

## Start LMX engine

In [ ]:
!/opt/lmx/bin/lmx-start-engine

## Import WMX library and Initialize a WMX3 API device

In [ ]:
# Import WMX3 API library
from WMX3ApiPython import *
# Import WMX3 Utils library for Python Notebook
from WMX3UtilPython import *

# When all the devices are done, the WMX3 engine will also terminate.	
wmx3Lib = WMX3Api()
wmx3LibCore = CoreMotion(wmx3Lib)

# Create a device.
wmx3Lib.CreateDevice('/opt/lmx', DeviceType.DeviceTypeNormal, INFINITE)

# Set wmx3Lib Name.
wmx3Lib.SetDeviceName('device')

# Initialize WMX3 Utils
IntitializeWMX3Util(wmx3Lib)

%matplotlib inline
%config InlineBackend.figure_format='retina'

## Set the device name and start communication

In [ ]:
# Set wmx3Lib Name.
ret = wmx3Lib.SetDeviceName('MotorControl')
if ret != ErrorCode.PyNone:
    CheckErrorCode("SetDeviceName", ret)
    
# Start Communication.
ret = wmx3Lib.StartCommunication(INFINITE)
if ret != ErrorCode.PyNone:
    CheckErrorCode("StartCommunication", ret)

In [ ]:
# Get created device state.
ret, devInfo = wmx3Lib.GetAllDevices()
if ret != ErrorCode.PyNone:
    CheckErrorCode("GetAllDevices", ret)
    
# Display the acquired device.
print(f'Device Id: {devInfo.GetDevices(0).id} Name: {devInfo.GetDevices(0).name}')

## Set servo ON ▶️

In [ ]:
wmx3LibCore.axisControl.SetServoOn(0, 1)
while True:
    # wmx3LibCore.GetStatus(CmStatus)
    ret, CmStatus = wmx3LibCore.GetStatus()
    if CmStatus.GetAxesStatus(0).servoOn:
        break
    sleep(0.1)

## 1. Absolute position command (StartPos)

Refer to the following documentation:

http://download.movensys.com:8222/doc/wmx3/en/html/api/classwmx3_api_1_1_motion.html#_CPPv4N7wmx3Api6Motion8StartPosEjP17TriggerPosCommand

In [ ]:
# Create a command value.
posCommand = Motion_PosCommand()

# Set position command parameters
posCommand.axis = 0
posCommand.profile.type = ProfileType.Trapezoidal
posCommand.profile.velocity = 10000
posCommand.profile.acc = 10000
posCommand.profile.dec = 10000

# Execute absolute position command to 10000
posCommand.target = 100000

logChannel, updater = StartLog()

# Rotate the motor at the specified speed.
err = wmx3LibCore.motion.StartPos(posCommand)

if err != ErrorCode.PyNone:
    errString = wmx3LibCore.ErrorToString(err)
    print("Failed to execute motion. Error=%d (%s)\n", err, errString)

# Wait for the motor to stop
err = wmx3LibCore.motion.Wait(0)
PauseLog(logChannel, updater)

### 1.1 Draw the position and velocity plots for absolute motion 📈

In [ ]:
plot_title = 'Absolute position command (StartPos)'
DrawLogPlot(plot_title, False)

## 2. Relative position command (StartMov)

Refer to the following documentation:

http://download.movensys.com:8222/doc/wmx3/en/html/api/classwmx3_api_1_1_motion.html#_CPPv4N7wmx3Api6Motion8StartMovEP10PosCommand

In [ ]:
# Execute relative position command of 10000
posCommand.target = 1000
posCommand.profile.velocity = 100
posCommand.profile.acc = 10
posCommand.profile.dec = 10

logChannel, updater = StartLog()

err = wmx3LibCore.motion.StartMov(posCommand)

if err != ErrorCode.PyNone:
    errString = wmx3LibCore.ErrorToString(err)
    print("Failed to execute motion. Error=%d (%s)\n", err, errString)

# Wait for the motor to stop
err = wmx3LibCore.motion.Wait(0)
PauseLog(logChannel, updater)

### 1.1 Draw the position and velocity plots for relative motion 📈

In [ ]:
plot_title = 'Relative position command (StartMov)'
DrawLogPlot(plot_title, False)

## Set servo OFF ⏹️

In [ ]:
wmx3LibCore.axisControl.SetServoOn(0, 0)
while True:
    # wmx3LibCore.GetStatus(CmStatus)
    ret, CmStatus = wmx3LibCore.GetStatus()
    if CmStatus.GetAxesStatus(0).servoOn:
        break
    sleep(0.1)

## Stop communication and close the WMX3 API device

In [ ]:
# ----------------------
# Stop Communication.
# ----------------------
wmx3Lib.StopCommunication(INFINITE)

# Close wmx3Lib.
wmx3Lib.CloseDevice()

## Stop LMX engine

In [ ]:
!/opt/lmx/bin/lmx-stop-engine